In [2]:
import pickle
import pandas as pd
import numpy as np
import datetime
import json

pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [3]:
from FormatEvents_Functions import *

In [4]:
home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

In [5]:
durations = ['0.5', '1', '2', '3', '6', '12', '24']

### UKCP18 data

### Join together lists for different ensemble members

In [39]:
events_props_dict_present = []
ems_present = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc012', 'bc013', 'bc015', 'bc016', 'bc017', 'bc018']
for em in ems_present:
    with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/event_props_dict_{em}.pickle", 'rb') as handle:
        one_events_props_dict_present = pickle.load(handle)    
    events_props_dict_present = events_props_dict_present + one_events_props_dict_present
    
## Join into one dataframe    
present = pd.DataFrame(events_props_dict_present)
present['Climate'] = 'Present'    

In [40]:
events_props_dict_future = []
# ems_future = ['bb195', 'bb192', 'bb198', 'bb208', 'bb225','bb222', 'bb201', 'bb204', 'bb216', 'bb219', 'bb211']
ems_future = ['bb192', 'bb208', 'bb225','bb222', 'bb201', 'bb204', 'bb216', 'bb219', 'bb211', 'bb189'] #bb195, #bb198
for em in ems_future:
    with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_props_dict_{em}.pickle", 'rb') as handle:
        one_events_props_dict_future = pickle.load(handle)    
    events_props_dict_future = events_props_dict_future + one_events_props_dict_future
    
## Join into one dataframe
future = pd.DataFrame(events_props_dict_future)
future['Climate'] = 'Future'

## Make a check on number of files (could shift this to the checking script)
NB - the method of searching on part1 doesnt work, because the filename only represents on of the files that is represented by that event


24529 is 19 * 1291 and is the number we expert with no part1s for one ensemble member.  
For 12 ems it becomes 24529 * 12 = 294348

In [8]:
# filtered_df = present[present['filename'].str.contains('part1')]

In [8]:
# Ensure that values are treated as lists. If any single numbers are not in a list, convert them to lists.
present['dur_for_which_this_is_amax'] = present['dur_for_which_this_is_amax'].apply(lambda x: x if isinstance(x, list) else [x])

# Flatten the lists and count occurrences of each number
all_numbers = [num for sublist in present['dur_for_which_this_is_amax'] for num in sublist]
number_counts = pd.Series(all_numbers).value_counts()

# Show the result
print(number_counts)

24     327523
12     297624
6      294422
0.5    294348
1      294348
2      294348
3      294348
dtype: int64


In [9]:
# Ensure that values are treated as lists. If any single numbers are not in a list, convert them to lists.
future['dur_for_which_this_is_amax'] = future['dur_for_which_this_is_amax'].apply(lambda x: x if isinstance(x, list) else [x])

# Flatten the lists and count occurrences of each number
all_numbers = [num for sublist in future['dur_for_which_this_is_amax'] for num in sublist]
number_counts = pd.Series(all_numbers).value_counts()

# Show the result
print(number_counts)

24     270114
12     247512
6      245335
0.5    245290
1      245290
2      245290
3      245290
dtype: int64


### Create one dataframe containing both present and future

In [41]:
df_long = pd.concat([present, future])

# Add D variable (day of year) and date
df_long['D'] = (df_long['theta'] * 365.25) / (2 * np.pi)
df_long['date'] = df_long.apply(lambda row: date_from_D(row['D'], row['year']), axis=1)
df_long['season'] = df_long['date'].apply(get_season)

### Check the number of files for each duration
NB: Number of files for 24h duration is longer due to compound events  
Checked this by filtering only rows with part0

### Remove entries which are less than 1.5 hours

In [12]:
# Convert lists to JSON strings before saving - not doing this, messed up formatting a bit
df_long['dur_for_which_this_is_amax'] = df_long['dur_for_which_this_is_amax'].apply(json.dumps)
df_long.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/all_events_characteristics_shortdurationskept.csv", index=False)

In [66]:
df_long = df_long[df_long['duration'] >=1.5]
present = present[present['duration'] >=1.5]
future = future[future['duration'] >=1.5]
# nan_rows = df_long[df_long['D50'].isna()]

In [15]:
df_long.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/all_events_characteristics.csv", index=False)

### NIMROD data

In [16]:
with open(home_dir +  f"ProcessedData/AMAX_Events/NIMROD_30mins/event_props_dict.pickle", 'rb') as handle:
    events_props_dict_nimrod = pickle.load(handle)

In [17]:
nimrod = pd.DataFrame(events_props_dict_nimrod)
# Add D variable (day of year) and date
nimrod['D'] = (nimrod['theta'] * 365.25) / (2 * np.pi)
nimrod['date'] = nimrod.apply(lambda row: date_from_D(row['D'], row['year']), axis=1)
nimrod['season'] = nimrod['date'].apply(get_season)

In [18]:
# Ensure that values are treated as lists. If any single numbers are not in a list, convert them to lists.
nimrod['dur_for_which_this_is_amax'] = nimrod['dur_for_which_this_is_amax'].apply(lambda x: x if isinstance(x, list) else [x])

# Flatten the lists and count occurrences of each number
all_numbers = [num for sublist in nimrod['dur_for_which_this_is_amax'] for num in sublist]
number_counts = pd.Series(all_numbers).value_counts()

# Show the result
print(number_counts)

24     19456
0.5    19343
12     14595
6      11434
2       8249
1       7667
3       6961
dtype: int64


In [19]:
# Ensure that values are treated as lists. If any single numbers are not in a list, convert them to lists.
nimrod['dur_for_which_this_is_amax'] = nimrod['dur_for_which_this_is_amax'].apply(lambda x: x if isinstance(x, list) else [x])

# Flatten the lists and count occurrences of each number
all_numbers = [num for sublist in nimrod['dur_for_which_this_is_amax'] for num in sublist]
number_counts = pd.Series(all_numbers).value_counts()

# Show the result
print(number_counts)

24     19456
0.5    19343
12     14595
6      11434
2       8249
1       7667
3       6961
dtype: int64


In [20]:
nimrod.to_csv(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/all_events_characteristics.csv", index=False)

## Create datasets with just events for each sampling duration

In [67]:
# for duration in durations:
#     df_long_this_dur = df_long[df_long['dur_for_which_this_is_amax'].apply(
#         lambda x: isinstance(x, list) and str(duration) in x or x == str(duration))]
#     print(duration, len(df_long_this_dur))

In [68]:
df_long_05hr = df_long[df_long['dur_for_which_this_is_amax'].apply(
    lambda x: isinstance(x, list) and str(0.5) in x or x == str(0.5))]

df_long_1hr = df_long[df_long['dur_for_which_this_is_amax'].apply(
    lambda x: isinstance(x, list) and str(1) in x or x == str(1))]

df_long_2hr = df_long[df_long['dur_for_which_this_is_amax'].apply(
    lambda x: isinstance(x, list) and str(2) in x or x == str(2))]

df_long_3hr = df_long[df_long['dur_for_which_this_is_amax'].apply(
    lambda x: isinstance(x, list) and str(3) in x or x == str(3))]

df_long_6hr = df_long[df_long['dur_for_which_this_is_amax'].apply(
    lambda x: isinstance(x, list) and str(6) in x or x == str(6))]

df_long_12hr = df_long[df_long['dur_for_which_this_is_amax'].apply(
    lambda x: isinstance(x, list) and str(12) in x or x == str(12))]

df_long_24hr = df_long[df_long['dur_for_which_this_is_amax'].apply(
    lambda x: isinstance(x, list) and str(24) in x or x == str(24))]

In [21]:
# for df in [df_long_24hr, df_long_12hr, df_long_6hr, df_long_3hr, df_long_2hr, df_long_1hr, df_long_05hr]:
#     print(len(df))

In [69]:
# vals, counts = np.unique(present['duration'], return_counts=True)
# my_df = pd.DataFrame({'values': vals, 'conts':counts})
# my_df.sort_values(by='conts', ascending=False)

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(present['D50'], bins=25)

# Create grouped results
### Group by gauge, climate

In [70]:
group_by_columns = ['Climate', 'gauge_num']
grouped_by_gauge_allevents = group_data_calc_means(df_long, group_by_columns)
grouped_by_gauge_allevents_changes = find_change_values_in_groups_new(grouped_by_gauge_allevents, group_by_columns, 'All')
grouped_by_gauge_allevents_changes.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_changes_allevents.csv", index=False)

### Group by season, gauge, climate

In [71]:
group_by_columns = ['Climate', 'gauge_num', 'season']
grouped_by_gauge_season_allevents = group_data_calc_means(df_long, group_by_columns)
grouped_by_gauge_season_allevents_changes = find_change_values_in_groups_new(grouped_by_gauge_season_allevents, group_by_columns, 'All')

grouped_by_gauge_season_allevents_changes.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_season_changes_allevents.csv", index=False)

In [98]:
1291 * 7

9037

In [99]:
total

,gauge_num,theta_mean_present,D_mean_present,R_present,D50_mean_present,D50_median_present,theta_mean_future,D_mean_future,R_future,D50_mean_future,D50_median_future,theta_mean_diff,D_mean_diff,R_diff,D50_mean_diff,D50_median_diff,sampling_duration
0,0,3.492009,202.995146,0.716161,51.787231,50.000000,3.652277,212.311765,0.665845,48.384561,50.000000,0.160268,9.316619,-0.050316,-3.402670,0.000000,0.500000
1,1,3.669006,213.284264,0.646034,46.225785,50.000000,4.117856,239.376471,0.562262,49.510553,50.000000,0.448849,26.092207,-0.083772,3.284768,0.000000,0.500000
2,2,3.636411,211.389474,0.659656,49.534355,50.000000,3.817106,221.893491,0.588810,49.686314,50.000000,0.180695,10.504017,-0.070846,0.151959,0.000000,0.500000
3,3,3.654827,212.460000,0.716496,47.729892,50.000000,3.785144,220.035503,0.529504,49.676508,50.000000,0.130317,7.575503,-0.186992,1.946617,0.000000,0.500000
4,4,3.589481,208.661376,0.681365,44.007283,45.833333,3.822357,222.198758,0.523637,49.619731,50.000000,0.232876,13.537382,-0.157729,5.612448,4.166667,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,1289,4.076974,237.000000,0.449279,49.761641,50.000000,3.930137,228.464115,0.366939,51.223931,50.000000,-0.146838,-8.535885,-0.082340,1.462290,0.000000,24.000000
1287,1290,3.795601,220.643382,0.323145,51.780593,51.065760,3.680511,213.953052,0.453697,51.031805,50.000000,-0.115090,-6.690331,0.130552,-0.748788,-1.065760,24.000000
1288,1291,3.499964,203.457627,0.324244,48.545873,48.368079,3.935950,228.802083,0.381297,46.338815,45.833333,0.435986,25.344456,0.057053,-2.207058,-2.534746,24.000000
1289,1292,3.759127,218.523077,0.280734,49.541361,50.000000,3.821431,222.144928,0.324373,50.571090,50.000000,0.062305,3.621851,0.043639,1.029729,0.000000,24.000000


## Create version of events with one event per gauge, per duration

In [72]:
group_by_columns = ['Climate', 'gauge_num']
grouped_by_gauge_samplingdur_05 = group_data_calc_means(df_long_05hr, group_by_columns)
grouped_by_gauge_samplingdur_1 = group_data_calc_means(df_long_1hr, group_by_columns)
grouped_by_gauge_samplingdur_2 = group_data_calc_means(df_long_2hr, group_by_columns)
grouped_by_gauge_samplingdur_3 = group_data_calc_means(df_long_3hr, group_by_columns)
grouped_by_gauge_samplingdur_6 = group_data_calc_means(df_long_6hr, group_by_columns)
grouped_by_gauge_samplingdur_12 = group_data_calc_means(df_long_12hr, group_by_columns)
grouped_by_gauge_samplingdur_24 = group_data_calc_means(df_long_24hr, group_by_columns)

In [75]:
group_by_columns = ['Climate', 'gauge_num', 'season']
grouped_by_gauge_samplingdur_season_05 = group_data_calc_means(df_long_05hr, group_by_columns)
grouped_by_gauge_samplingdur_season_1 = group_data_calc_means(df_long_1hr, group_by_columns)
grouped_by_gauge_samplingdur_season_2 = group_data_calc_means(df_long_2hr, group_by_columns)
grouped_by_gauge_samplingdur_season_3 = group_data_calc_means(df_long_3hr, group_by_columns)
grouped_by_gauge_samplingdur_season_6 = group_data_calc_means(df_long_6hr, group_by_columns)
grouped_by_gauge_samplingdur_season_12 = group_data_calc_means(df_long_12hr, group_by_columns)
grouped_by_gauge_samplingdur_season_24 = group_data_calc_means(df_long_24hr, group_by_columns)

In [76]:
group_by_columns = ['Climate', 'gauge_num']
grouped_by_gauge_samplingdur_05_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_05, group_by_columns, float(0.5))
grouped_by_gauge_samplingdur_1_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_1, group_by_columns, 1)
grouped_by_gauge_samplingdur_2_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_2, group_by_columns, 2)
grouped_by_gauge_samplingdur_3_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_3, group_by_columns,3)
grouped_by_gauge_samplingdur_6_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_6, group_by_columns,6)
grouped_by_gauge_samplingdur_12_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_12, group_by_columns, 12)
grouped_by_gauge_samplingdur_24_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_24, group_by_columns, 24)

In [77]:
group_by_columns = ['Climate', 'gauge_num', 'season']
grouped_by_gauge_samplingdur_season_05_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_season_05, group_by_columns, float(0.5))
grouped_by_gauge_samplingdur_season_1_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_season_1, group_by_columns, 1)
grouped_by_gauge_samplingdur_season_2_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_season_2, group_by_columns, 2)
grouped_by_gauge_samplingdur_season_3_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_season_3, group_by_columns,3)
grouped_by_gauge_samplingdur_season_6_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_season_6, group_by_columns,6)
grouped_by_gauge_samplingdur_season_12_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_season_12, group_by_columns, 12)
grouped_by_gauge_samplingdur_season_24_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_season_24, group_by_columns, 24)

In [78]:
group_by_columns = ['Climate', 'gauge_num']
grouped_by_gauge_samplingdur_05_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_05, group_by_columns, float(0.5))
grouped_by_gauge_samplingdur_1_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_1, group_by_columns, 1)
grouped_by_gauge_samplingdur_2_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_2, group_by_columns, 2)
grouped_by_gauge_samplingdur_3_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_3, group_by_columns,3)
grouped_by_gauge_samplingdur_6_changes  = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_6, group_by_columns,6)
grouped_by_gauge_samplingdur_12_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_12, group_by_columns, 12)
grouped_by_gauge_samplingdur_24_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_24, group_by_columns, 24)

In [79]:
total = pd.concat([grouped_by_gauge_samplingdur_05_changes, grouped_by_gauge_samplingdur_1_changes, 
                  grouped_by_gauge_samplingdur_2_changes, grouped_by_gauge_samplingdur_3_changes,
                  grouped_by_gauge_samplingdur_6_changes, grouped_by_gauge_samplingdur_12_changes,
                  grouped_by_gauge_samplingdur_24_changes])

In [97]:
total

,gauge_num,theta_mean_present,D_mean_present,R_present,D50_mean_present,D50_median_present,theta_mean_future,D_mean_future,R_future,D50_mean_future,D50_median_future,theta_mean_diff,D_mean_diff,R_diff,D50_mean_diff,D50_median_diff,sampling_duration
0,0,3.492009,202.995146,0.716161,51.787231,50.000000,3.652277,212.311765,0.665845,48.384561,50.000000,0.160268,9.316619,-0.050316,-3.402670,0.000000,0.500000
1,1,3.669006,213.284264,0.646034,46.225785,50.000000,4.117856,239.376471,0.562262,49.510553,50.000000,0.448849,26.092207,-0.083772,3.284768,0.000000,0.500000
2,2,3.636411,211.389474,0.659656,49.534355,50.000000,3.817106,221.893491,0.588810,49.686314,50.000000,0.180695,10.504017,-0.070846,0.151959,0.000000,0.500000
3,3,3.654827,212.460000,0.716496,47.729892,50.000000,3.785144,220.035503,0.529504,49.676508,50.000000,0.130317,7.575503,-0.186992,1.946617,0.000000,0.500000
4,4,3.589481,208.661376,0.681365,44.007283,45.833333,3.822357,222.198758,0.523637,49.619731,50.000000,0.232876,13.537382,-0.157729,5.612448,4.166667,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,1289,4.076974,237.000000,0.449279,49.761641,50.000000,3.930137,228.464115,0.366939,51.223931,50.000000,-0.146838,-8.535885,-0.082340,1.462290,0.000000,24.000000
1287,1290,3.795601,220.643382,0.323145,51.780593,51.065760,3.680511,213.953052,0.453697,51.031805,50.000000,-0.115090,-6.690331,0.130552,-0.748788,-1.065760,24.000000
1288,1291,3.499964,203.457627,0.324244,48.545873,48.368079,3.935950,228.802083,0.381297,46.338815,45.833333,0.435986,25.344456,0.057053,-2.207058,-2.534746,24.000000
1289,1292,3.759127,218.523077,0.280734,49.541361,50.000000,3.821431,222.144928,0.324373,50.571090,50.000000,0.062305,3.621851,0.043639,1.029729,0.000000,24.000000


In [80]:
total_season = pd.concat([grouped_by_gauge_samplingdur_season_05_changes, grouped_by_gauge_samplingdur_season_1_changes, 
                  grouped_by_gauge_samplingdur_season_2_changes, grouped_by_gauge_samplingdur_season_3_changes,
                  grouped_by_gauge_samplingdur_season_6_changes, grouped_by_gauge_samplingdur_season_12_changes,
                  grouped_by_gauge_samplingdur_season_24_changes])

In [ ]:
# group_by_columns = ['Climate', 'gauge_num', 'duration', 'season']
# grouped_by_gauge_samplingdur_season = group_data_calc_means(df_long, group_by_columns)
# grouped_by_gauge_samplingdur_season_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur_season, group_by_columns)

# group_by_columns = ['Climate', 'gauge_num', 'duration']
# grouped_by_gauge_samplingdur = group_data_calc_means(df_long, group_by_columns)
# grouped_by_gauge_samplingdur_changes = find_change_values_in_groups_new(grouped_by_gauge_samplingdur, group_by_columns)

In [83]:
# grouped_by_gauge_samplingdur_season_changes.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_samplingdur_season_changes_new.csv", index=False)
total.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_changes_bydur.csv", index=False)

total_season.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_season_changes_bydur.csv", index=False)